## 🗒️ Introdução

Este notebook tem como objetivo realizar a limpeza e padronização do dataset
Financial Sample, garantindo qualidade, consistência e confiabilidade dos dados
antes das análises exploratórias e de negócio.


In [53]:
import pandas as pd

In [54]:
# Carregamento dos dados brutos
# Visualização inicial para entendimento da estrutura
finances = pd.read_csv("../../database/financial_sample.csv")
finances.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Montana,High,200.0,5,350,70000.0,9800.0,60200.0,52000.0,8200.0,5/1/2014,5,May,2014
1,Small Business,Germany,Carretera,Low,214.0,3,300,64200.0,1284.0,62916.0,53500.0,9416.0,10/1/2013,10,October,2013
2,Small Business,Germany,VTT,Low,214.0,250,300,64200.0,1284.0,62916.0,53500.0,9416.0,10/1/2013,10,October,2013
3,Midmarket,Canada,Paseo,Low,218.0,10,15,3270.0,130.8,3139.2,2180.0,959.2,9/1/2014,9,September,2014
4,Government,Germany,Paseo,High,241.0,10,20,4820.0,482.0,4338.0,2410.0,1928.0,10/1/2014,10,October,2014


In [55]:
# Colunas antes da padronização
display(finances.columns)

# %%
# Padronização dos nomes das colunas (snake_case)
finances.columns = (
    finances.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
)

# Colunas depois da padronização
display(finances.columns)

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       'Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')

Index(['segment', 'country', 'product', 'discount_band', 'units_sold',
       'manufacturing_price', 'sale_price', 'gross_sales', 'discounts',
       'sales', 'cogs', 'profit', 'date', 'month_number', 'month_name',
       'year'],
      dtype='object')

In [56]:
# Verificação dos tipos de dados
finances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   segment              700 non-null    object 
 1   country              700 non-null    object 
 2   product              700 non-null    object 
 3   discount_band        647 non-null    object 
 4   units_sold           700 non-null    float64
 5   manufacturing_price  700 non-null    int64  
 6   sale_price           700 non-null    int64  
 7   gross_sales          700 non-null    float64
 8   discounts            700 non-null    float64
 9   sales                700 non-null    float64
 10  cogs                 700 non-null    float64
 11  profit               700 non-null    float64
 12  date                 700 non-null    object 
 13  month_number         700 non-null    int64  
 14  month_name           700 non-null    object 
 15  year                 700 non-null    int

In [57]:
# Conversão de tipos de dados

# Conversão da coluna de data
finances["date"] = pd.to_datetime(finances["date"])

# Conversão de colunas inteiras
finances["units_sold"] = finances["units_sold"].astype(int)

In [58]:
# Conversão de colunas numéricas
numeric_cols = [
    "manufacturing_price", "sale_price",
    "gross_sales", "discounts",
    "sales", "cogs", "profit"
]

finances[numeric_cols] = finances[numeric_cols].astype(float)

In [59]:
# Verificação de valores nulos
finances.isna().sum()

segment                 0
country                 0
product                 0
discount_band          53
units_sold              0
manufacturing_price     0
sale_price              0
gross_sales             0
discounts               0
sales                   0
cogs                    0
profit                  0
date                    0
month_number            0
month_name              0
year                    0
dtype: int64

In [62]:
# Tratamento da coluna discount_band
# Valores nulos representam vendas sem desconto
finances["discount_band"] = (
    finances["discount_band"]
        .str.strip()
        .fillna("Not Informed")
        .str.title()
)

In [63]:
# Padronização global de colunas textuais
# Remove espaços em branco no início e no fim de todas as colunas categóricas
text_cols = finances.select_dtypes(include="object").columns
finances[text_cols] = finances[text_cols].apply(lambda x: x.str.strip())

In [64]:
# Validação exploratória de regras de negócio
# As regras abaixo apresentam inconsistências no dataset original:
# - sales = gross_sales - discounts
# - profit = sales - cogs
#
# As correções e decisões sobre essas regras serão tratadas na camada de SQL.

(finances["sales"] == finances["gross_sales"] - finances["discounts"]).value_counts()
(finances["profit"] == finances["sales"] - finances["cogs"]).value_counts()

True     526
False    174
Name: count, dtype: int64

In [66]:
# Salvamento do dataset limpo e padronizado
finances.to_excel(
    "../../database/financial.xlsx",
    index=False
)